# 第10章: 事前学習済み言語モデル（GPT型）

本章では、GPT型（Transformerのデコーダ型）の事前学習済みモデルを利用して、言語生成、評判分析器（ポジネガ分類器）の構築、ファインチューニング、強化学習などに取り組む。

## 90. 次単語予測

“The movie was full of"に続くトークン（トークン列ではなく一つのトークンであることに注意せよ）として適切なもの上位10個と、その確率（尤度）を求めよ。ただし、言語モデルへのプロンプトがどのようなトークン列に変換されたか、確認せよ。

In [5]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import torch.nn.functional as F

# 1. トークナイザーとモデルのロード
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.eval()

# 2. プロンプトをトークナイズ
prompt = "The movie was full of"
input_ids = tokenizer.encode(prompt, return_tensors="pt")  # shape: [1, seq_len]

# トークンID列を確認
print("Token IDs:", input_ids[0].tolist())
print("Tokens:", [tokenizer.decode([tid]) for tid in input_ids[0]],"\n")

# 3. モデルに入力して出力（次のトークンのロジットを取得）
with torch.no_grad():
    outputs = model(input_ids)
    logits = outputs.logits  # shape: [1, seq_len, vocab_size]

# 最後のトークンの次のトークンの確率分布
next_token_logits = logits[0, -1, :]  # shape: [vocab_size]

# 4. 確率化（ソフトマックス）
probs = F.softmax(next_token_logits, dim=0)

# 5. 上位10個トークンを取得
topk = torch.topk(probs, k=10)
topk_probs = topk.values.tolist()
topk_indices = topk.indices.tolist()

print("Top 10 next tokens and their probabilities:")
for idx, prob in zip(topk_indices, topk_probs):
    print(f"Token: '{tokenizer.decode([idx])}' (ID: {idx}), Probability: {prob:.4f}")


Token IDs: [464, 3807, 373, 1336, 286]
Tokens: ['The', ' movie', ' was', ' full', ' of'] 

Top 10 next tokens and their probabilities:
Token: ' jokes' (ID: 14532), Probability: 0.0219
Token: ' great' (ID: 1049), Probability: 0.0186
Token: ' laughs' (ID: 22051), Probability: 0.0115
Token: ' bad' (ID: 2089), Probability: 0.0109
Token: ' surprises' (ID: 24072), Probability: 0.0107
Token: ' references' (ID: 10288), Probability: 0.0105
Token: ' fun' (ID: 1257), Probability: 0.0100
Token: ' humor' (ID: 14733), Probability: 0.0074
Token: ' "' (ID: 366), Probability: 0.0074
Token: ' the' (ID: 262), Probability: 0.0067


## 91. 続きのテキストの予測

“The movie was full of"に続くテキストを複数予測せよ。このとき、デコーディングの方法や温度パラメータ（temperature）を変えながら、予測される複数のテキストの変化を観察せよ。

In [9]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.eval()

prompt = "The movie was full of"
input_ids = tokenizer.encode(prompt, return_tensors="pt")

# GPT-2にはpad_token_idがない → eos_token_id (50256) を代用する
pad_token_id = tokenizer.eos_token_id
attention_mask = input_ids.ne(pad_token_id).long()  # すべて1（パディングしていない）になるがOK

def generate_text(method="greedy", temperature=1.0, top_k=50, top_p=0.9, num_beams=3, max_length=50):
    generation_args = {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "pad_token_id": pad_token_id,
        "max_length": max_length
    }

    if method == "greedy":
        generation_args["do_sample"] = False
    elif method == "beam":
        generation_args.update({"do_sample": False, "num_beams": num_beams, "early_stopping": True})
    elif method == "top-k":
        generation_args.update({"do_sample": True, "top_k": top_k, "temperature": temperature})
    elif method == "top-p":
        generation_args.update({"do_sample": True, "top_p": top_p, "temperature": temperature})
    else:
        raise ValueError("Unsupported decoding method")

    outputs = model.generate(**generation_args)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# 動作確認
print("=== Top-p Sampling (p=0.9, temp=1.0) ===")   #常に一番確率の高いトークンを選択。決定的な結果で多様性がない。
print(generate_text(method="top-p", top_p=0.9, temperature=1.0))

print("\n=== Greedy Decoding ===")    #複数候補（ビーム幅）を保持しつつ、全体の尤度が高いものを選ぶ。Greedyよりは多様性が出るが、似た表現に偏ることが多い
print(generate_text(method="greedy"))

print("\n=== Beam Search (num_beams=5) ===")    #次トークンの上位k個からランダムに選ぶ。多様性が出やすい
print(generate_text(method="beam", num_beams=5))

print("\n=== Top-k Sampling (k=50, temp=1.0) ===")    #確率の累積がpになるまでのトークンからランダムに選ぶ。動的に候補数が変わり、多様性と品質のバランスが良い
print(generate_text(method="top-k", top_k=50, temperature=1.0))

print("\n=== Top-p Sampling (p=0.9, temp=0.5) ===")   #分布を鋭くして「確率の高いトークンをより選びやすく」
print(generate_text(method="top-p", top_p=0.9, temperature=0.5))

print("\n=== Top-p Sampling (p=0.9, temp=1.5) ===")   #分布を平坦にして「多様性が増す」
print(generate_text(method="top-p", top_p=0.9, temperature=1.5))


=== Top-p Sampling (p=0.9, temp=1.0) ===
The movie was full of good laughs. Some of it was funny because of the characters, but all the time I think we're very proud of this team that we have. We know the way they play, we know how they play, and we

=== Greedy Decoding ===
The movie was full of jokes and jokes about how the movie was a joke. It was a joke about how the movie was a joke. It was a joke about how the movie was a joke. It was a joke about how the movie was a

=== Beam Search (num_beams=5) ===
The movie was full of jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes and jokes

=== Top-k Sampling (k=50, temp=1.0) ===
The movie was full of spoilers, and a lot of people thought that this all fit the movie. Well, in the end, I found out what happened, and they did a great job on the ending, and if I did it again thi

## 92. 予測されたテキストの確率を計算

“The movie was full of"に続くテキストを予測し、生成された各単語の尤度を表示せよ（生成されるテキストが長いと出力が読みにくくなるので、適当な長さで生成を打ち切るとよい）。

In [10]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import torch.nn.functional as F

# モデルとトークナイザーの準備
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.eval()

# 初期プロンプト
prompt = "The movie was full of"
input_ids = tokenizer.encode(prompt, return_tensors="pt")
generated_ids = input_ids.clone()

# 最大生成トークン数
max_new_tokens = 10

# 尤度を記録
log_probs = []

for _ in range(max_new_tokens):
    # モデル出力
    with torch.no_grad():
        outputs = model(input_ids=generated_ids)
        logits = outputs.logits  # (1, seq_len, vocab_size)

    # 最新トークンに対する確率分布
    next_token_logits = logits[0, -1, :]  # (vocab_size,)
    probs = F.softmax(next_token_logits, dim=-1)
    log_prob = F.log_softmax(next_token_logits, dim=-1)

    # Greedyで次トークンを選ぶ
    next_token_id = torch.argmax(probs).unsqueeze(0)
    log_prob_value = log_prob[next_token_id].item()
    log_probs.append(log_prob_value)

    # 生成文を更新
    generated_ids = torch.cat([generated_ids, next_token_id.unsqueeze(0)], dim=1)

# 結果表示
generated_tokens = generated_ids[0]
decoded = tokenizer.decode(generated_tokens, skip_special_tokens=True)
new_tokens = generated_tokens[len(input_ids[0]):]

print("=== 生成結果 ===")
print(decoded)

print("\n=== 各トークンと尤度 ===")
for token_id, lp in zip(new_tokens, log_probs):
    token_str = tokenizer.decode(token_id)
    print(f"{token_str!r:>15} : log_prob = {lp:.4f}")


=== 生成結果 ===
The movie was full of jokes and jokes about how the movie was a joke

=== 各トークンと尤度 ===
       ' jokes' : log_prob = -3.8216
         ' and' : log_prob = -1.2405
       ' jokes' : log_prob = -2.3177
       ' about' : log_prob = -1.5820
         ' how' : log_prob = -2.3054
         ' the' : log_prob = -2.4694
       ' movie' : log_prob = -3.3129
         ' was' : log_prob = -1.2162
           ' a' : log_prob = -2.6930
        ' joke' : log_prob = -1.7515


## 93. パープレキシティ

適当な文を準備して、事前学習済み言語モデルでパープレキシティを測定せよ。例えば、

+ The movie was full of surprises
+ The movies were full of surprises
+ The movie were full of surprises
+ The movies was full of surprises

の4文に対して、パープレキシティを測定して観察せよ（最後の2つの文は故意に文法的な間違いを入れた）。

In [11]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import torch.nn.functional as F
import math

# モデル・トークナイザーの準備
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.eval()

def calculate_perplexity(sentence):
    encodings = tokenizer(sentence, return_tensors="pt")
    input_ids = encodings.input_ids
    with torch.no_grad():
        outputs = model(**encodings, labels=input_ids)
        loss = outputs.loss
    perplexity = torch.exp(loss)
    return perplexity.item()

# テスト文（文法正誤入り）
sentences = [
    "The movie was full of surprises",     # 正しい
    "The movies were full of surprises",   # 正しい
    "The movie were full of surprises",    # 主語-動詞の一致誤り
    "The movies was full of surprises"     # 主語-動詞の一致誤り
]

# 各文に対するパープレキシティを表示
print("=== パープレキシティ測定結果 ===")   #パープレキシティ：ある言語モデルにとって、ある文がどれだけ「予測しやすいか」の尺度
for sentence in sentences:
    ppl = calculate_perplexity(sentence)
    print(f"{sentence:<40} : PPL = {ppl:.2f}")


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


=== パープレキシティ測定結果 ===
The movie was full of surprises          : PPL = 99.35
The movies were full of surprises        : PPL = 126.48
The movie were full of surprises         : PPL = 278.88
The movies was full of surprises         : PPL = 274.66


## 94. チャットテンプレート

"What do you call a sweet eaten after dinner?"という問いかけに対する応答を生成するため、チャットテンプレートを適用し、言語モデルに与えるべきプロンプトを作成せよ。また、そのプロンプトに対する応答を生成し、表示せよ。

In [21]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# モデルとトークナイザーの準備
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.eval()

# チャットテンプレートに基づくプロンプト作成
prompt = (
    "<|system|>\nYou are a helpful assistant.\n"
    "<|user|>\nWhat do you call a sweet eaten after dinner?\n"
    "<|assistant|>\n"
)

# トークン化
input_ids = tokenizer.encode(prompt, return_tensors="pt")

# 応答生成（Top-pサンプリング）
output_ids = model.generate(
    input_ids,
    max_length=input_ids.shape[1] + 30,
    do_sample=True,
    top_p=0.9,
    temperature=0.8,
    pad_token_id=tokenizer.eos_token_id
)

# 応答部分を抽出
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
response = generated_text.split("<|assistant|>\n")[-1].strip()

# 出力
print("=== 応答 ===")
print(response)


=== 応答 ===
What do you call a sweet eaten after dinner?

<|your|>
What do you call a sweet eaten after dinner?


## 95. マルチターンのチャット

問題94で生成された応答に対して、追加で"Please give me the plural form of the word with its spelling in reverse order."と問いかけたときの応答を生成・表示せよ。また、その時に言語モデルに与えるプロンプトを確認せよ。

In [22]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# モデルとトークナイザーの読み込み
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.eval()

# チャット風プロンプトの構築
prompt = (
    "<|system|>\nYou are a helpful assistant.\n"
    "<|user|>\nWhat do you call a sweet eaten after dinner?\n"
    "<|assistant|>\nWhat do you call a sweet eaten after dinner?\n"
    "<|user|>\nPlease give me the plural form of the word with its spelling in reverse order.\n"
    "<|assistant|>\n"
)

# トークナイズ
input_ids = tokenizer.encode(prompt, return_tensors="pt")

# 応答生成
output_ids = model.generate(
    input_ids,
    max_length=input_ids.shape[1] + 40,
    do_sample=True,
    top_p=0.9,
    temperature=0.8,
    pad_token_id=tokenizer.eos_token_id
)

# 応答部分の抽出
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
response = generated_text.split("<|assistant|>\n")[-1].strip()

# 表示
print("=== 与えたプロンプト ===")
print(prompt)
print("\n=== モデルの応答 ===")
print(response)


=== 与えたプロンプト ===
<|system|>
You are a helpful assistant.
<|user|>
What do you call a sweet eaten after dinner?
<|assistant|>
What do you call a sweet eaten after dinner?
<|user|>
Please give me the plural form of the word with its spelling in reverse order.
<|assistant|>


=== モデルの応答 ===
What do you call a sweet eaten after dinner?
<|user|>
Please give me the plural form of the word with its spelling in reverse order.
<|assistant|>


## 96. プロンプトによる感情分析

事前学習済み言語モデルで感情分析を行いたい。テキストを含むプロンプトを事前学習済み言語モデルに与え、（ファインチューニングは行わずに）テキストのポジネガを予測するという戦略で、[SST-2](https://dl.fbaipublicfiles.com/glue/data/SST-2.zip)の開発データにおける正解率を測定せよ。

## 97. 埋め込みに基づく感情分析

事前学習済み言語モデルでテキストをベクトルで表現（エンコード）し、そのベクトルにフィードフォワード層を通すことで極性ラベルを予測するモデルを学習せよ。

## 98. ファインチューニング

問題96のプロンプトに対して、正解の感情ラベルをテキストの応答として返すように事前学習済みモデルをファインチューニングせよ。

## 99. 選好チューニング

問題96のプロンプトに対して、正解の感情ラベルを含むテキストを望ましい応答、間違った感情ラベルを含むテキストを望ましくない応答として、事前学習済み言語モデルを選好チューニング (preference tuning) を実施せよ。選好チューニングのアルゴリズムとしては、近傍方策最適化 (PPO: Proximal Policy Optimization) や直接選好最適化 (DPO: Direct Preference Optimization) などが考えられる。
